In [6]:
import pandas as pd
import d00_utils.read_log as dr
# read log file to lines and
# read log file
logfile_folder_path = os.path.join(
    os.path.expanduser("~"),
    'simulation',
    'lammps_process',
    'data',
    '01_raw',
    'Link to lmp_run',
    'test',
    '20200511',
    'f_5e6',
)
logfile_path = os.path.join(logfile_folder_path, "log.lammps")
dr.get_log_variable_dic_fromlog_variable(logfile_folder_path)
# get previous log_variable
folder_path_list_last_to_initial = [logfile_folder_path]
log_variable = dr.get_log_variable_dic_fromlog_variable(logfile_folder_path)
log_variable_dic_list_last_to_initial = [log_variable]
parent_folder_path = os.path.abspath(os.path.join(logfile_folder_path, os.pardir))
while log_variable['rst_from'] != '0' and os.path.isfile(os.path.join(parent_folder_path, 'log.lammps')):
    log_variable = dr.get_log_variable_dic_fromlog_variable(parent_folder_path)
    log_variable_dic_list_last_to_initial.append(
        log_variable
    )
    folder_path_list_last_to_initial.append(
        parent_folder_path
    )
    parent_folder_path = os.path.abspath(os.path.join(parent_folder_path, os.pardir))
log_variable_dic_list = log_variable_dic_list_last_to_initial[::-1]
if log_variable_dic_list[0]['rst_from'] != '0':
    sys.exit('log_variable wrong')

n_simu_total = len(log_variable_dic_list)
folder_path_list_initial_to_last = folder_path_list_last_to_initial[::-1]

log_variable_dic_list[0]['rst_from']


KeyboardInterrupt: 

In [3]:
n_simu_total

NameError: name 'n_simu_total' is not defined

In [ ]:
[logdiclist, n_simu_total, logfile, folder_path_list_initial_to_last] = dr.get_dumped_variable(logfile_folder_path, '/home/ic6413/simulation/lammps_process/data/02_intermediate/test.pickle')

In [ ]:
# calculate previous time for every log
restart_time = 0
for n in range(n_simu_total):
    if n == 0:
        logdiclist[n]["previous_time"] = restart_time
    else:
        for n_line1, line in enumerate(line_log_list[n-1]):
            if len(line) >= 2:
                if line.split()[0]=="Step" and line.split()[1]=="Time":
                    break
        
        for n_line2, line in enumerate(line_log_list[n-1]):
            if len(line) >= 2:
                if line.split()[0]=="Loop" and line.split()[1]=="time":           
                    break

        for n_line, line in enumerate(line_log_list[n-1]):
            if n_line>n_line1 and n_line<n_line2:
                if line.split()[0]==str(logdiclist[n]['rst_from']):
                    restart_time += float(line.split()[1])
        logdiclist[n]["previous_time"] = restart_time

# count the time no rotation
rotate_start_time = 0
for i in range(n_simu_total):
    if int(logdiclist[i]["rst_from"]) == 0:
        previous_time_step = 0
    else:
        previous_time_step = float(logdiclist[i-1]["ts"])
    rotate_start_time += previous_time_step*int(logdiclist[i]["rst_from"])
    if logdiclist[i]["ifrotate"] == "yes" and float(logdiclist[i]["Sa"]) != 0:
        logdiclist[i]["rotate_start_time"] = rotate_start_time
        break
    else:
        logdiclist[i]["rotate_start_time"] = None # have not rotate

for i in range(n_simu_total):    
    logdiclist[i]["rotate_start_time"] = rotate_start_time